# Binary Star Fitting: Curve Fit (Cont.)

Continuing last week's exercise with fitting the binary star systems velocity variation, we now explore the power of emcee as a more efficient approach to sample the possible solution space. We'll explore:

1. **Theoretical Background**: What does emcee do differently from scipy curvefit
2. **Curve Fitting**: Using `emcee` in action and examine the results

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import emcee
import corner
from astropy import units as u
from astropy.constants import G
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# let's load the data
url = "https://drive.google.com/uc?export=download&id=1hmYypOQxt7ZJ1-JxYurqU7tnsNGBMMDj" #binary_measurements_observed.csv file
binary_data_inc = pd.read_csv(url)
# binary_data_inc


In [ ]:
t_obs = binary_data_inc['time']
v_r_obs = binary_data_inc['vlos']
sigma_obs_array = binary_data_inc['error']

## 2. Bayesian Analysis with emcee

Now let's use `emcee` to perform a Bayesian analysis. This will explore the entire parameter space and reveal all possible solutions, not just the local minimum found by `curve_fit`.


In [ ]:
# define the models for calculating the radial velocities of stars
def solve_kepler_equation(E, e):
    """Solve Kepler's equation: M = E - e*sin(E)"""
    M = E - e * np.sin(E)
    return M

def get_true_anomaly(E, e):
    """Calculate true anomaly from eccentric anomaly"""
    f = 2 * np.arctan(np.sqrt((1 + e) / (1 - e)) * np.tan(E / 2))
    return f

def radial_velocity_model(t, P, e, omega, T0, K, gamma):
    """
    Calculate radial velocity for a binary star system
    
    Parameters:
    -----------
    t : array_like
        Time array
    P : float
        Orbital period in days
    e : float
        Eccentricity (0-1)
    omega : float
        Argument of periastron in radians
    T0 : float
        Time of periastron passage
    K : float
        Velocity amplitude in km/s
    gamma : float
        Systemic velocity in km/s
    
    Returns:
    --------
    v_r : array_like
        Radial velocity in km/s
    """
    # Mean anomaly
    M = 2 * np.pi * (t - T0) / P
    
    # Solve for eccentric anomaly using Newton-Raphson
    E = M.copy()  # Initial guess
    for _ in range(10):  # Max iterations
        E_new = E - (E - e * np.sin(E) - M) / (1 - e * np.cos(E))
        if np.allclose(E, E_new, rtol=1e-10):
            break
        E = E_new
    
    # True anomaly
    f = get_true_anomaly(E, e)
    
    # Radial velocity
    v_r = gamma + K * (np.cos(omega + f) + e * np.cos(omega))
    
    return v_r

In [ ]:
# Define the log-likelihood function
def log_likelihood(theta, t, v_r, sigma):
    """
    Log-likelihood function for the radial velocity model
    """
    P, e, omega, T0, K, gamma = theta
    
    # Calculate model predictions
    v_r_model = radial_velocity_model(t, P, e, omega, T0, K, gamma)
    
    # Calculate log-likelihood (assuming Gaussian errors)
    log_like = -0.5 * np.sum(((v_r - v_r_model) / sigma)**2 + np.log(2 * np.pi * sigma**2))
    
    return log_like

# Define the log-prior function
def log_prior(theta):
    """
    Log-prior function with broad, uninformative priors
    """
    P, e, omega, T0, K, gamma = theta
    
    # Broad priors
    if (10 < P < 500 and 
        0 < e < 0.99 and 
        0 < omega < 2*np.pi and 
        0 < T0 < 100 and 
        5 < K < 50 and 
        -100 < gamma < 100):
        return 0.0  # Uniform prior
    
    return -np.inf  # Outside prior range


# Define the log-posterior function
def log_posterior(theta, t, v_r, sigma):
    """
    Log-posterior function
    """
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    
    return lp + log_likelihood(theta, t, v_r, sigma)

# Set up the MCMC sampler
nwalkers = 50  # Number of walkers
ndim = 6       # Number of parameters
nsteps = 2000  # Number of steps

# Initial positions for walkers (scattered around the parameter space)
initial_positions = np.array([
    np.random.uniform(20, 490, nwalkers),      # P
    np.random.uniform(0.1, 0.8, nwalkers),     # e
    np.random.uniform(0, 2*np.pi, nwalkers),   # omega
    np.random.uniform(0, 80, nwalkers),        # T0
    np.random.uniform(10, 40, nwalkers),       # K
    np.random.uniform(-90, 90, nwalkers)       # gamma
]).T

print(f"Setting up MCMC with {nwalkers} walkers, {nsteps} steps")
print(f"Initial positions spread across parameter space")

# Create the sampler
sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_posterior, 
    args=(t_obs, v_r_obs, sigma_obs_array)
)



In [ ]:
# Run the MCMC
print("Running MCMC...")
sampler.run_mcmc(initial_positions, nsteps, progress=True)

# Get the samples (discard burn-in)
burn_in = 500
samples = sampler.get_chain(discard=burn_in, flat=True)

print(f"\nMCMC completed!")
print(f"Discarded {burn_in} steps as burn-in")
print(f"Final sample size: {len(samples)} points")



In [ ]:
# Calculate acceptance fraction
acceptance_fraction = np.mean(sampler.acceptance_fraction)
print(f"Average acceptance fraction: {acceptance_fraction:.3f}")

# Plot the chain evolution
fig, axes = plt.subplots(ndim, 1, figsize=(12, 10), sharex=True)
for i, param_name in enumerate(param_names):
    axes[i].plot(sampler.get_chain()[:, :, i], alpha=0.3)
    axes[i].axvline(burn_in, color='r', linestyle='--', alpha=0.7)
    axes[i].set_ylabel(param_name)
    if i == ndim - 1:
        axes[i].set_xlabel('Step')

plt.suptitle('MCMC Chain Evolution')
plt.tight_layout()
plt.show()


## 2.1 Analyze the Posterior Distribution

Now let's analyze the posterior distribution to see what the Bayesian analysis reveals about the parameter space.


In [ ]:
# Calculate summary statistics
param_medians = np.median(samples, axis=0)
param_errors = np.std(samples, axis=0)
param_16th = np.percentile(samples, 16, axis=0)
param_84th = np.percentile(samples, 84, axis=0)

print("Bayesian Analysis Results:")
print("Parameter | Median | 16th %ile | 84th %ile")
print("-" * 70)

for i, name in enumerate(param_names):
    median_val = param_medians[i]
    low_val = param_16th[i]
    high_val = param_84th[i]
    
    if name == 'omega':
        print(f"{name:8s} | {median_val:6.3f} | {low_val:8.3f} | {high_val:8.3f}")
    else:
        print(f"{name:8s} | {median_val:6.3f} | {low_val:8.3f} | {high_val:8.3f}")

# Create corner plot
fig = corner.corner(
    samples, 
    labels=param_names,
    quantiles=[0.16, 0.5, 0.84],
    show_titles=True,
    title_kwargs={"fontsize": 12}
)

plt.suptitle('Posterior Distribution', fontsize=16)
# plt.tight_layout()
plt.show()



In [ ]:
# Plot individual parameter distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, name in enumerate(param_names):
    ax = axes[i]
    
    # Histogram of samples
    ax.hist(samples[:, i], bins=50, alpha=0.7, density=True, color='skyblue')
    
    # Median and confidence intervals
    ax.axvline(param_medians[i], color='black', linestyle='-', linewidth=2, 
               label='Median')
    ax.axvline(param_16th[i], color='black', linestyle=':', alpha=0.7)
    ax.axvline(param_84th[i], color='black', linestyle=':', alpha=0.7)
    
    ax.set_xlabel(name)
    ax.set_ylabel('Density')
    ax.set_title(f'{name} Posterior Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3)

# plt.tight_layout()
plt.show()



In [ ]:
# Calculate the number of distinct solutions
print("\nAnalyzing posterior for multiple solutions...")

# Look for bimodal distributions (especially in omega)
omega_samples = samples[:, 2]  # omega parameter
omega_hist, omega_bins = np.histogram(omega_samples, bins=100)
omega_centers = (omega_bins[:-1] + omega_bins[1:]) / 2

# Find peaks in the omega distribution
from scipy.signal import find_peaks
peaks, _ = find_peaks(omega_hist, height=np.max(omega_hist)*0.1, distance=10)

print(f"Found {len(peaks)} peaks in omega distribution")
for i, peak in enumerate(peaks):
    peak_omega = omega_centers[peak]
    print(f"Peak {i+1}: omega = {peak_omega:.3f} rad ({np.degrees(peak_omega):.1f}°)")

### Key Takeaways:

1. **Incomplete Data Challenges**: When observations are sparse or incomplete, multiple solutions can fit the data equally well. This is a common problem in astronomy.

2. **Initial Guess Sensitivity**: The `curve_fit` method is sensitive to initial parameter guesses. With incomplete data, different starting points can lead to different (but equally valid) solutions.

3. **Bayesian Advantages**: The `emcee` approach explores the entire parameter space and reveals all possible solutions, not just local minima.

4. **Uncertainty Quantification**: Bayesian methods provide proper uncertainty estimates that account for parameter correlations and degeneracies.

5. **Prior Information**: The choice of priors in Bayesian analysis can significantly influence results, especially with limited data.

### Real-World Applications:

- **Exoplanet Detection**: Radial velocity surveys often have incomplete phase coverage
- **Binary Star Studies**: Limited observing windows due to weather, telescope scheduling
- **Asteroid Orbits**: Sparse observations over long time periods
- **Galaxy Dynamics**: Limited spatial coverage in velocity field measurements

### Best Practices:

1. **Always check multiple initial guesses** when using optimization methods
2. **Use Bayesian methods** when dealing with incomplete or sparse data
3. **Consider parameter degeneracies** and their physical implications
4. **Plan observations** to break degeneracies when possible
5. **Report uncertainties** that reflect the true parameter space
